In [175]:
import miner2
import miner2.miner as miner
import os, sys, datetime
import dill
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

%matplotlib inline

GDC_download_folder = '/Users/user/Projects/melanoma/data/melanoma_data'
results = '/Users/user/Projects/melanoma/Results/'

dill.load_session(results+'subtype_discovery.dill')

#Variables
    #num_cores = 4
    #min_number_genes = 6
    #min_correlation = 0.2
    #
    #expressionData (Dataframe of filtered genes against cases)
    #coexpressionDict (Revised list of clusters formated as a dictionary)
    #mechanistic_output (Dictionary of dictionaries???)
    #referenceDictionary = mechanistic_output
    
def kaplanMeier(df, groups=None, statusCol='status', durationCol='duration'):
    allSurvival = df.copy()
#   survivalGroups = np.array(groups)
    
    if groups == None:
        durations = []
        event_observed = []
    
        for status, days in allSurvival.values:
            if status == 'Dead':
                durations.append(int(days/365))
                event_observed.append(1)
            elif status == 'Alive':
                durations.append(max(df.loc[:, 'duration'].values)/365+1)
                event_observed.append(0)
    
        kmf = KaplanMeierFitter()
        kmf.fit(durations, event_observed, label='KM Estimate')

        kmf.plot(ci_show=False)
    
    elif groups is not None:
        for subtype in groups:
            tempDf = allSurvival.loc[subtype]
            print(subtype)
            print(tempDf)
            
            durations = []
            event_observed = []

            for status, days in allSurvival.values:
                if status == 'Dead':
                    durations.append(int(days/365))
                    event_observed.append(1)
                elif status == 'Alive':
                    durations.append(max(df.loc[:, 'duration'].values)/365+1)
                    event_observed.append(0)

            kmf = KaplanMeierFitter()
            kmf = kmf.fit(durations, event_observed, label='KM Estimate')
            
            
            kmf.plot(ci_show=False)
    
    return kmf
#             kmEstimate = kaplanMeier(groupdDf)
    
pd.set_option('display.max_rows', 500)

In [318]:
survivalSKCM = pd.read_csv("/Users/user/Projects/melanoma/data/clinical.cart.2019-08-14/clinical.tsv", sep='\t', index_col=1,header=0)
survivalDfSKCM = survivalSKCM.iloc[:,7:9]
survivalDfSKCM.columns = ["status", "duration"]
survivalDfSKCM = survivalDfSKCM.loc[(survivalDfSKCM.status == 'Dead') | (survivalDfSKCM.status == 'Alive')]
survivalDfSKCM.loc[survivalDfSKCM.status == 'Alive', 'duration'] = 0
survivalDfSKCM['duration'] = survivalDfSKCM['duration'].astype(int)

indexConversion = pd.read_csv("/Users/user/Projects/melanoma/data/gdc_sample_sheet.2019-08-14.tsv", sep='\t', index_col=5, header=0)

trimId = []
for fileId in indexConversion['File Name']:
    trimId.append(fileId.split('.')[0])
indexConversion['File Name'] = trimId
repeatsRemove = ['TCGA-ER-A19T-06A', 'TCGA-ER-A2NF-06A', 'TCGA-D3-A1QA-07A', 'TCGA-GN-A4U8-11A']
indexConversion = indexConversion.loc[~indexConversion['Sample ID'].isin(repeatsRemove)]
indexConversion = indexConversion.loc[survivalDfSKCM.index, :]
survivalDfSKCM.index = indexConversion['File Name']

In [350]:
goodCases = survivalDfSKCM.index.intersection(groups[1])
group1survival = survivalDfSKCM.loc[goodCases]
group1survival

,status,duration
b3eaff31-b803-47f9-acca-469053767155,Dead,368
3320f3dc-67aa-440c-a817-24f5a04f1a14,Dead,1691
9943770b-b3e2-42c9-a286-85b558de7da3,Dead,4062
c62cc001-e373-466e-9305-32420ac84858,Dead,447
0498d3e9-bf60-48ff-bf34-d9725718c576,Dead,1524
0934e651-4cea-4450-b95e-25ee99e276f4,Dead,369
61f6cb8d-5b6e-4385-bd7e-3a75e3c94ff5,Dead,549
0a3f430a-7e5e-4e68-92cc-f767da7e361c,Dead,650
b147912f-7aa8-4d78-bdbc-7028fb73560d,Dead,907
0d2ee969-027b-4a7e-be4d-c44aec59c23b,Dead,619


In [351]:
l = list(indexConversion.index)
repeats = set([x for x in l if l.count(x) > 1])
repeatsIndex = indexConversion.loc[repeats, :]
# repeatsIndex